Model should be trained multiple times with different random seeds to get the best model.

In [ ]:
pip install tensorflow-gpu==1.15.0

     |████████████████████████████████| 411.5 MB 7.2 kB/s 
     |████████████████████████████████| 50 kB 8.9 MB/s 
     |████████████████████████████████| 503 kB 96.1 MB/s 
     |████████████████████████████████| 3.8 MB 70.2 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=1b93f5427596dc4368671f3a698e630809f98ee3d4e6fd038d40466644292341
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succe

In [ ]:
!pip install keras==2.2.4

     |████████████████████████████████| 312 kB 8.2 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.0 requires keras<2.8,>=2.7.0rc0, but you have keras 2.2.4 which is incompatible.
tensorflow 2.7.0 requires tensorboard~=2.6, but you have tensorboard 1.15.0 which is incompatible.
tensorflow 2.7.0 requires tensorflow-estimator<2.8,~=2.7.0rc0, but you have tensorflow-estimator 1.15.1 which is incompatible.


In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-xiba1f3l
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-xiba1f3l
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=918f218497b8f67389c6e9106991e779720ceeff165037703381a93d3a366ae4
  Stored in directory: /tmp/pip-ephem-wheel-cache-5rt_zec8/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# LOAD DATA

In [ ]:
from tensorflow import keras 
import pandas as pd
import numpy as np
from ast import literal_eval
from nltk.tokenize import TweetTokenizer
import spacy
from copy import deepcopy
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.initializers import Constant
from nltk.corpus import stopwords

Using TensorFlow backend.


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
pd.options.display.max_rows

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [ ]:

data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
dev = pd.read_csv('dev.csv')

text_data = data['content'].values
spans_data = data['index_spans'].apply(literal_eval)
lbl_data = [1 if len(s) > 0 else 0 for s in spans_data]

text_test = test['content'].values
spans_test = test['index_spans'].apply(literal_eval)
lbl_test = [1 if len(s) > 0 else 0 for s in spans_test]

text_dev = dev['content'].values
spans_dev = dev['index_spans'].apply(literal_eval)
lbl_dev = [1 if len(s) > 0 else 0 for s in spans_dev]

In [ ]:
tokenizer = TweetTokenizer()

def tokenize_text(text):
  return tokenizer.tokenize(text)

def toxic_word(span, text):
  i = 0
  token = []
  a = 0
  word = []

  while (i < (len(span) - 1)):
      if (span[i] != (span[i+1]-1)):
          token.append(span[a:(i+1)])
          a = i + 1
      elif i == (len(span) - 2):
          token.append(span[a:i+2])
      i = i + 1
  for t in token:
      word.append(text[t[0]:(t[len(t)-1])+1])
  return word
def span_convert(text_data, spans):
    MAX_LEN = 0
    token_labels = []

    for i in range(0, len(text_data)):
        token_labels.append(toxic_word(spans[i], text_data[i]))

    lst_seq = []
    for i in range(0, len(text_data)):
        token = tokenize_text(text_data[i])
        if len(token) > MAX_LEN:
            MAX_LEN = len(token)
            
        seq = np.zeros(len(token), dtype=int)
        for j in range(0, len(token)):
            for t in token_labels[i]:
                if token[j] in tokenize_text(t):
                    seq[j] = 1
        lst_seq.append(seq)     

    return (token_labels, lst_seq)
def span_retrived(text_data, spans):
    token_labels = []

    for i in range(0, len(text_data)):
        token_labels.append(toxic_word(spans[i], text_data[i]))
    
    return token_labels
def decoding(text_data, encoding_text, prediction):
    test = [[idx2word[i] for i in row] for row in encoding_text]

    lst_token = []

    for t in range(0, len(test)):
        yy_pred = []
        for i in range(0, len(test[t])):
            if prediction[t][i] == 1:
                yy_pred.append(test[t][i])
        lst_token.append(yy_pred)

    lis_idx = []
    for i in range(0, len(text_data)):
        idx = []
        for t in lst_token[i]:
            index = text_data[i].find(t)
            idx.append(index)
            for j in range(1, len(t)):
                index = index + 1
                idx.append(index)
        lis_idx.append(idx)

    return lis_idx

In [ ]:
data['token'], data['seq'] =  span_convert(text_data, spans_data)
dev['token'], dev['seq'] =  span_convert(text_dev, spans_dev)
test['token'], test['seq'] =  span_convert(text_test, spans_test)
data.head(10)
train = deepcopy(data)

,Unnamed: 0,level_0,index,content,index_spans,token,seq
0,0,2051,104,À thế bạn ý hát bài j jui có ai giới thiệu cho mình đc ko. Lâu jui mình chẳng nghe bài hát mới nào cả,[],[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,1,2087,140,Thầy đang nghĩ về cô em xinh tươi này ...,[],[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,2,2357,410,"Ủa tứ quý có chặt đôi heo đc đâu, lộ tứ quý rồi nhaaa hahhahaha t cho m thúi Trần Thị Linh Ánh Bui Thi My Ngoc Thanh Hà Thanh Thảo Thùy Trang Lê Thị Oanh Hoàng Linhc",[],[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,3,2588,641,"tốt, cứ giá cao đi -_- chứ bh nông dân cũng mua đc xe thì k biết đường nó còn ra cái gì",[],[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,4,3854,1907,Mấy cái quan bệnh tật của thời đại,[],[],"[0, 0, 0, 0, 0, 0, 0, 0]"
5,5,2565,618,Mặt nó lại ngáo 🤣,[],[],"[0, 0, 0, 0, 0]"
6,6,617,617,Ngu cmm.,"[0, 1, 2, 3, 4, 5, 6]",[Ngu cmm],"[1, 1, 0]"
7,7,918,918,ai coi trần dần và ngô kỷ là kẻ ngu ngóc...,"[29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]",[kẻ ngu ngóc],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]"
8,8,2118,171,Nhưng mà khi tỏ tình xong rồi bất bại vừa mất ng mình thương vừa mất luôn bạn 🙂,[],[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
9,9,3170,1223,tộc caaaa,[],[],"[0, 0]"


# EVALUATION METRIC

In [ ]:
import sys
import os
import os.path
from scipy.stats import sem
import numpy as np
from ast import literal_eval

def f1(predictions, gold):
    """
    F1 (a.k.a. DICE) operating on two lists of offsets (e.g., character).
    >>> assert f1([0, 1, 4, 5], [0, 1, 6]) == 0.5714285714285714
    :param predictions: a list of predicted offsets
    :param gold: a list of offsets serving as the ground truth
    :return: a score between 0 and 1
    """
    if len(gold) == 0:
        return 1. if len(predictions) == 0 else 0.
    if len(predictions) == 0:
        return 0.
    predictions_set = set(predictions)
    gold_set = set(gold)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)


def evaluate(pred, gold):
    """
    Based on https://github.com/felipebravom/EmoInt/blob/master/codalab/scoring_program/evaluation.py
    :param pred: file with predictions
    :param gold: file with ground truth
    :return:
    """
    # # read the predictions
    # pred_lines = pred.readlines()
    # # read the ground truth
    # gold_lines = gold.readlines()

    pred_lines = pred
    gold_lines = gold

    # only when the same number of lines exists
    if (len(pred_lines) == len(gold_lines)):
        data_dic = {}
        for n, line in enumerate(gold_lines):
            parts = line.split('\t')
            if len(parts) == 2:
                data_dic[int(parts[0])] = [literal_eval(parts[1])]
            else:
                raise ValueError('Format problem for gold line %d.', n)

        for n, line in enumerate(pred_lines):
            parts = line.split('\t')
            if len(parts) == 2:
                if int(parts[0]) in data_dic:
                    try:
                        data_dic[int(parts[0])].append(literal_eval(parts[1]))
                    except ValueError:
                        # Invalid predictions are replaced by a default value
                        data_dic[int(parts[0])].append([])
                else:
                    raise ValueError('Invalid text id for pred line %d.', n)
            else:
                raise ValueError('Format problem for pred line %d.', n)

        # lists storing gold and prediction scores
        scores = []
        for id in data_dic:
            if len(data_dic[id]) == 2:
                gold_spans = data_dic[id][0]
                pred_spans = data_dic[id][1]
                scores.append(f1(pred_spans, gold_spans))
            else:
                sys.exit('Repeated id in test data.')

        return (np.mean(scores), sem(scores))


# Word Embedding

In [ ]:
# Read embedding
word_dict = []
embeddings_index = {}
f = open('word2vec_vi_words_100dims.txt')
for line in f:
    values = line.split(' ')
    word = values[0] 
    word_dict.append(word)
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('GloVe data loaded')

GloVe data loaded


In [ ]:
words = word_dict
num_words = len(words)

# Dictionary word:index pair
# word is key and its value is corresponding index
word_to_index = {w : i + 2 for i, w in enumerate(words)}
word_to_index["UNK"] = 1
word_to_index["PAD"] = 0

# Dictionary lable:index pair
idx2word = {i: w for w, i in word_to_index.items()}

In [ ]:
# first create a matrix of zeros, this is our embedding matrix
embedding_dim = 100
max_len = 10
max_feature = 1000
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_to_index.items():
    if i > max_feature:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

In [ ]:
 # mapping for token cases
case2Idx = {'1': 1, '0': 0}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')  # identity matrix used 

char2Idx = {"PADDING": 0, "UNKNOWN": 1}
for c in " 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,-_()[]{}!?:;#'\"/\\%$`&=*+@^~|<>":
    char2Idx[c] = len(char2Idx)

In [ ]:
from sklearn.model_selection import train_test_split

y = data['seq']
X = data['content']

y_test = test['seq']
X_test = test['content']

y_dev = dev['seq']
X_dev = dev['content']

X_train = X
y_train = y

In [ ]:
tknzr2 = TweetTokenizer()

def custom_tokenizer(text_data):
    text_data = text_data.lower()
    return tknzr2.tokenize(text_data)

def encoding(X, y, isTest = True):
    sentences = []
    
    for t in X:
        sentences.append(custom_tokenizer(t))

    X = []
    for s in sentences:
        sent = []
        for w in s:
            try:
                w = w.lower()
                sent.append(word_to_index[w])
            except:
                sent.append(word_to_index["UNK"])
        X.append(sent)
           
    X = pad_sequences(maxlen = max_len, sequences = X, padding = "post", value = word_to_index["PAD"])

    if isTest:
        y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=word_to_index["PAD"])
        y = to_categorical(y, num_classes=2)
    else:
        y = None

    return (X,y)

In [ ]:
X1, y1 = encoding(X_train, y_train)
X2, y2 = encoding(X_dev, y_dev)
X3, y3 = encoding(X_test, y_test)

In [ ]:
# BiLSTM - CRF 
from keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional, Flatten, Dropout

from keras.models import Model, Input
from keras_contrib.layers import CRF
from keras.utils.vis_utils import plot_model

import warnings
warnings.filterwarnings("ignore")

# from keras.metrics import BinaryAccuracy, Precision, Recall, AUC

input = Input(shape = (max_len,))
model = Embedding(input_dim=num_words+2,
                    output_dim=embedding_dim,
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length=max_len,
                    trainable=True)(input)

model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units = max_len, return_sequences=True, recurrent_dropout=0.1))(model)
model = TimeDistributed(Dense(max_len, activation="relu"))(model)
crf = CRF(2)  
out = crf(model)  # output

model = Model(input, out)
model.compile(optimizer="adam", loss=crf.loss_function, metrics=['accuracy'])

model.summary()

plot_model(model,to_file="bilstm-crf.pdf",show_shapes=True,show_layer_names=True)




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 100)           158751000 
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 100)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 10, 20)            8880      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 10)            210       
____________________________________________________________

In [ ]:
from keras.callbacks import ModelCheckpoint
import warnings
warnings.filterwarnings("ignore")

checkpointer = ModelCheckpoint(filepath = 'model_detection_19_word.h5',
                       verbose = 0,
                       mode = 'auto',
                       save_best_only = True,
                       monitor='val_loss')

model.fit(X1, np.array(y1), batch_size=512, epochs=4, validation_data=(X2, y2), callbacks=[checkpointer])


Train on 3894 samples, validate on 488 samples
Epoch 1/4
3894/3894 [==============================] - 1s 177us/step - loss: 0.5223 - acc: 0.8888 - val_loss: 0.3817 - val_acc: 0.5721
Epoch 2/4
3894/3894 [==============================] - 1s 181us/step - loss: 0.3129 - acc: 0.8888 - val_loss: 0.2514 - val_acc: 0.8525
Epoch 3/4
3894/3894 [==============================] - 1s 172us/step - loss: 0.2206 - acc: 0.8888 - val_loss: 0.2151 - val_acc: 0.8857
Epoch 4/4
3894/3894 [==============================] - 1s 176us/step - loss: 0.1961 - acc: 0.8888 - val_loss: 0.2073 - val_acc: 0.8850


In [ ]:
y_pred = model.predict(X3)
y_pred = np.argmax(y_pred, axis=-1)
y_test_true = np.argmax(y3, -1)
# y_pred = [[i for i in row] for row in y_pred]

In [ ]:
raw_y = decoding(X_test, X3, y_pred)


error df

In [ ]:
token_predict, seq_predict = span_convert(text_test, raw_y)

In [ ]:
error = pd.DataFrame()
error['True'] = test['token']
error['Pred'] = token_predict

# PREDICT

In [ ]:
def tokenize(text, pos):
    tokens = text.split()
    alignment = []
    start = 0
    for t in tokens:
        res = text.find(t, start)
        alignment.append(pos[res:res + len(t)])
        start = res + len(t)
    assert len(tokens) == len(alignment)
    return tokens, alignment
def y_true(data):
  index_true = []
  data['index_spans'] = data['index_spans'].apply(literal_eval)
  for i in range(len(data)):
    text = data['content'][i]
    pos = [i for i in range(len(text))]
    df_point = pd.DataFrame()
    df_point['spans'] = pos
    df_point['spans'] = 0
    if not data['index_spans'][i]:
      index_true.append(list(df_point['spans']))
    else:
      for j in data['index_spans'][i]:
        df_point['spans'][j] = 1
      index_true.append(list(df_point['spans']))
  return index_true
def y_pred(test, error):
  index_pred = []
  for dt in range(len(error)):
    value_predict_i = error['Pred'][dt]
    text = test['content'][dt]
    pos = [i for i in range(len(text))]
    tokens, alignment = tokenize(text, pos)
    df_point = pd.DataFrame()
    df_point['spans'] = pos
    df_point['spans'] = 0
    for j in range(len(tokens)):
      if tokens[j] in value_predict_i:
        for ali in alignment[j]:
            df_point['spans'][ali] = 1
    index_pred.append(list(df_point['spans']))
  return index_pred
true = y_true(test)
pred = y_pred(test, error)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
scores_f1_macro = []
scores_f1_micro = []
scores_precision_macro = []
scores_precision_micro = []
scores_recall_macro = []
scores_recall_micro = []

for i in range(len(true)):
  score_macro = precision_recall_fscore_support(true[i], pred[i], average='macro')
  score_micro = precision_recall_fscore_support(true[i], pred[i], average='micro')

  scores_f1_macro.append(score_macro[2])
  scores_f1_micro.append(score_micro[2])
  scores_precision_macro.append(score_macro[0])
  scores_precision_micro.append(score_micro[0])
  scores_recall_macro.append(score_macro[1])
  scores_recall_micro.append(score_micro[1])

In [ ]:
scores = pd.DataFrame()
scores['F1-micro'] = [np.mean(scores_f1_micro)]
scores['F1-macro'] = [np.mean(scores_f1_macro)]
scores['Precision-macro'] = [np.mean(scores_precision_macro)]
scores['Precision-micro'] = [np.mean(scores_precision_micro)]
scores['Recall-macro'] = [np.mean(scores_recall_macro)]
scores['Recall-micro'] = [np.mean(scores_recall_micro)]

# Save

In [ ]:
scores.to_csv('score.csv')

In [ ]:
error.to_csv('error.csv')

In [ ]:
model.save('model.h5')
